In [ ]:
%run load_environment.ipynb

In [ ]:
import boto3
import os
import pyspark.sql.functions
from pyspark.sql import SparkSession

In [ ]:
appName = "NemesysNotebook"
try:
    appName = APP_NAME
except:
    pass

In [ ]:
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
         .getOrCreate()
)
         # .enableHiveSupport()

In [ ]:
#--------------------------------------------------------------------------
# Configurar o acesso ao Azure Data Lake (Blob)
#--------------------------------------------------------------------------
spark.conf.set(f'fs.azure.account.auth.type.{STORAGE_ACCOUNT}.dfs.core.windows.net', 'SharedKey')
spark.conf.set(f'fs.azure.account.key.{STORAGE_ACCOUNT}.dfs.core.windows.net', STORAGE_KEY)
#--------------------------------------------------------------------------
# Configurar o acesso ao S3
#--------------------------------------------------------------------------
spark.conf.set('fs.s3a.endpoint', S3_URL)
spark.conf.set('fs.s3a.access.key', S3_ACCESS_KEY)
spark.conf.set('fs.s3a.secret.key', S3_SECRET_KEY)
spark.conf.set("spark.sql.debug.maxToStringFields", "100")
spark.conf.set("fs.s3a.path.style.access", "true")
spark.conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.conf.set("fs.s3a.connection.ssl.enabled", "true")

In [ ]:
#--------------------------------------------------------------------------
# Checar se Delta Table existe
#--------------------------------------------------------------------------
def delta_exists(delta_path, tier, table):
    if delta_path.starts_with("abfss://"):
        url = f'DefaultEndpointsProtocol=https;AccountName={storage_account};AccountKey={storage_key};EndpointSuffix=core.windows.net'
        blob = BlobClient.from_connection_string(conn_str=url, container_name=BLOB_CONTAINER, blob_name=f'{LAKEHOUSE_PATH}/bronze/{db}_{topic}')
        return blob.exists()
    elif delta_path.starts_with("s3a"):
        return ""
    else:
        return path.exists(delta_path)
#--------------------------------------------------------------------------
# Retornar os caminhos da tabela
#--------------------------------------------------------------------------    
def paths(topico: str, db: str, tier: str):
    delta_path = f'abfss://{BLOB_CONTAINER}@{STORAGE_ACCOUNT}.dfs.core.windows.net/{LAKEHOUSE_PATH}/{tier}/{db}_{topico}'
    checkpoint_path = f'abfss://{BLOB_CONTAINER}@{STORAGE_ACCOUNT}.dfs.core.windows.net/{LAKEHOUSE_PATH}/{tier}/checkpoint/kafka/{db}_{topico}'

    return delta_path, checkpoint_path
#--------------------------------------------------------------------------
# Carregar uma delta table e registrar como temporária
#--------------------------------------------------------------------------
def loadAndRegister(table:str, db:str, tier:str = "bronze"):
    delta_path, _ = paths(table, db, tier)
    df = spark.read.format("delta").load(delta_path)
    df.createOrReplaceTempView(f"{tier}_{table}")
    return df